In [1]:
from pandas.io import sql
import tqdm
import numpy as np
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from time import time, sleep
import time
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
import random

In [2]:
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - ====== WebDriver manager ======


[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Trying to download new driver from https://chromedriver.storage.googleapis.com/88.0.4324.96/chromedriver_mac64.zip
[WDM] - Driver has been saved in cache [/Users/admin/.wdm/drivers/chromedriver/mac64/88.0.4324.96]


# Gathering URLS for Premier League

- First, I scrape the URLS of every game from one of the leagues over the past 4 seasons. 
- Then I scrape all the data from each game.
- The reason I am showing only one leagues is because I would scrape one league at a time. This was due to if there was a break it would be easier to fix and sort out. 
- After Scraping all the matches from each league I would concatenate them 


In [5]:
prem_urls = []

In [6]:
driver.get('https://fbref.com/en/comps/9/1631/schedule/2017-2018-Premier-League-Scores-and-Fixtures')

try:
    driver.find_element_by_link_text("Agree").click()
except:
    pass
try: 
    button_1 = driver.find_element_by_class_name('css-flk0bs')
    button_1.click()
except:
    pass

for i in range(4):

    table = driver.find_element_by_id('content')

    match_reports = table.find_elements_by_link_text('Match Report')

    for link in match_reports:
        prem_urls.append(link.get_attribute('href'))
    try:
        driver.find_element_by_link_text('Next Season').click()
    except:
        print('End of Season')


End of Season
End of Season


In [19]:
attack_df_list = []
defence_df_list = []
gk_df_list = []



meta = {'Date': [], 
        'Competition': [], 
        'Home_manager': [], 
        'Away_manager': [], 
        'Ref' : [], 
        'Attendance': [], 
        'home_captain': [], 
        'Home_team': [], 
        'Away_team': [], 
        'Game_id': [], 
        'away_captain': [], 
        'home_conceded': [],
        'away_conceded': [],
        'home_xG': [], 
        'away_xG' : []}
# Here I am creating a game ID 
game_id = 1

for url in tqdm(prem_urls[0:5]):
    try:
        driver.get(url)
        try:
            driver.find_element_by_link_text("Agree").click()
        except:
            pass
        try: 
            button_1 = driver.find_element_by_class_name('css-flk0bs')
            button_1.click()
        except:
            pass

        try:
            home_team = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/div[1]/strong/a').text
        except:
            print('home_team')
        try:
            away_team = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[2]/div[1]/strong/a').text
        except:
            print('away_team')
        try:
            date = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[3]/div[1]/strong/a').text
        except:
            print('date')
        try:
            attendance = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[3]/div[5]').text
        except:
            print('attendance')

        competition = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[3]/div[2]/a').text
        
        try:
            away_captain = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[2]/div[6]/a').text
            home_captain = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/div[6]/a').text
        except:
            print(f"Check game for away captain {url}")
            home_captain = np.nan
            away_captain = np.nan

                
        away_manager = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[2]/div[5]').text
        home_manager = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/div[5]').text
        competition = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[3]/div[2]/a').text
        try:
            ref = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[3]/div[7]/small/span[1]').text
        except:
            ref = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[3]/div[6]/small/span[1]').text
        away_conceded = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/div[2]/div[1]').text
        home_conceded = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[2]/div[2]/div[1]').text
        home_xG = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/div[2]/div[2]').text
        away_xG = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[2]/div[2]/div[2]').text


        att_df_home = pd.read_html(url)[3]
        att_df_home['game_id'] = game_id
        att_df_home['Team'] = home_team

        att_df_away = pd.read_html(url)[10]
        att_df_away['game_id'] = game_id
        att_df_away['Team'] = away_team

        def_df_home = pd.read_html(url)[6]
        def_df_home['game_id'] = game_id
        def_df_home['Team'] = home_team

        def_df_away = pd.read_html(url)[13]
        def_df_away['game_id'] = game_id
        def_df_away['Team'] = away_team

        gk_df_home = pd.read_html(url)[9]
        gk_df_home['game_id'] = game_id
        gk_df_home['Team'] = home_team

        gk_df_away = pd.read_html(url)[16]
        gk_df_away['game_id'] = game_id
        gk_df_away['Team'] = away_team


        attack_df_list.append(att_df_home)
        attack_df_list.append(att_df_away)
        defence_df_list.append(def_df_home)
        defence_df_list.append(def_df_away)
        gk_df_list.append(gk_df_home)
        gk_df_list.append(gk_df_away)


        meta['Date'].append(date)
        meta['Competition'].append(competition)
        meta['Home_manager'].append(home_manager)
        meta['Away_manager'].append(away_manager)
        meta['Attendance'].append(attendance)
        meta['Ref'].append(ref)
        meta['Game_id'].append(game_id)


        meta['Away_team'].append(away_team)
        meta['Home_team'].append(home_team)
        meta['home_captain'].append(home_captain)
        meta['away_captain'].append(away_captain)
        meta['away_xG'].append(away_xG)
        meta['home_xG'].append(home_xG)
        meta['home_conceded'].append(home_conceded)
        meta['away_conceded'].append(away_conceded)




        game_id +=1

    except:
        print('Error With Url: ', url)


attack_df_2 = pd.concat(attack_df_list)

defence_df_2 = pd.concat(defence_df_list)

gk_df_2 = pd.concat(gk_df_list)

meta_df = pd.DataFrame(meta)    


100%|██████████| 5/5 [00:51<00:00, 10.20s/it]


In [20]:
attack_df_2

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0  \
                 Player                  #             Nation   
0   Alexandre Lacazette                9.0             fr FRA   
1         Danny Welbeck               23.0            eng ENG   
2          Theo Walcott               14.0            eng ENG   
3            Mesut Özil               11.0             de GER   
4          Granit Xhaka               29.0             ch SUI   
..                  ...                ...                ...   
10         Alfie Mawson                6.0            eng ENG   
11   Federico Fernández               33.0             ar ARG   
12        Kyle Naughton               26.0            eng ENG   
13     Łukasz Fabiański                1.0             pl POL   
14           14 Players                NaN                NaN   

   Unnamed: 3_level_0 Unnamed: 4_level_0 Unnamed: 5_level_0 Performance      \
                  Pos                Age                Min         Gls Ast   
0               FW,LW             26-075               90.0           1   0   
1               AM,RW             26-258               75.0           1   0   
2                  RW             28-148               15.0           0   0   
3                  AM             28-300               90.0           0   0   
4                  DM             24-318               90.0           0   2   
..                ...                ...                ...         ...  ..   
10                 CB             23-205               90.0           0   0   
11              CB,RB             28-172               90.0           0   0   
12              RB,RM             28-274               90.0           0   0   
13                 GK             32-116               90.0           0   0   
14                NaN                NaN              990.0           0   0   

             ... Passes                 Carries      Dribbles     game_id  \
   PK PKatt  ...    Cmp  Att  Cmp% Prog Carries Prog     Succ Att           
0   0     0  ...     19   24  79.2    1      24    2        1   1       1   
1   0     0  ...     16   21  76.2    2      25    6        3   4       1   
2   0     0  ...      1    2  50.0    0       2    1        0   0       1   
3   0     0  ...     69   85  81.2    3      84   15        2   3       1   
4   0     0  ...     83  102  81.4   15      89   13        0   0       1   
.. ..   ...  ...    ...  ...   ...  ...     ...  ...      ...  ..     ...   
10  0     0  ...     34   41  82.9    0      30    1        0   0       5   
11  0     0  ...     40   51  78.4    2      44    2        0   0       5   
12  0     0  ...     38   49  77.6    4      27    1        0   0       5   
13  0     0  ...     21   32  65.6    0      20    0        0   0       5   
14  0     0  ...    292  383  76.2   21     304   17        6   7       5   

            Team  
                  
0        Arsenal  
1        Arsenal  
2        Arsenal  
3        Arsenal  
4        Arsenal  
..           ...  
10  Swansea City  
11  Swansea City  
12  Swansea City  
13  Swansea City  
14  Swansea City  

[149 rows x 34 columns]

In [24]:
meta_df

,Date,Competition,Home_manager,Away_manager,Ref,Attendance,home_captain,Home_team,Away_team,Game_id,away_captain,home_conceded,away_conceded,home_xG,away_xG
0,"Friday August 11, 2017",Premier League,Manager: Arsène Wenger,Manager: Craig Shakespeare,Mike Dean (Referee),"Attendance: 59,387",Petr Čech,Arsenal,Leicester City,1,Wes Morgan,3,4,2.1,1.6
1,"Saturday August 12, 2017",Premier League,Manager: Marco Silva,Manager: Jürgen Klopp,Anthony Taylor (Referee),"Attendance: 20,407",Heurelho Gomes,Watford,Liverpool,2,Jordan Henderson,3,3,2.0,3.0
2,"Saturday August 12, 2017",Premier League,Manager: Tony Pulis,Manager: Eddie Howe,Robert Madley (Referee),"Attendance: 25,011",Jake Livermore,West Bromwich Albion,Bournemouth,3,Simon Francis,0,1,1.2,0.4
3,"Saturday August 12, 2017",Premier League,Manager: Ronald Koeman,Manager: Mark Hughes,Niel Swarbrick (Referee),"Attendance: 39,045",Phil Jagielka,Everton,Stoke City,4,Ryan Shawcross,0,1,0.4,0.3
4,"Saturday August 12, 2017",Premier League,Manager: Mauricio Pellegrino,Manager: Paul Clement,Mike Jones (Referee),"Attendance: 31,447",Steven Davis,Southampton,Swansea City,5,Leon Britton,0,0,2.1,0.4


## Changing Column Names
Due to the fact that the scraped data frame has multi-index columns I needed to remove this and rename the columns. This is in order to convert to SQL as it cannot have repeated column names. 

In [21]:
def column_replacer(df):
    df.columns = df.columns.droplevel()
    count = 1
    new_cols = []
    for column in df.columns:
        if list(df.columns).count(column) > 1:
            new_cols.append(f'{column}_{count}')
            count +=1
        else:
            new_cols.append(column)
    df.columns = new_cols
    df.rename(columns={df.columns[-1]: 'team', df.columns[-2]: 'game_id'}, inplace=True)
    
    

In [22]:
column_replacer(attack_df_2)

column_replacer(defence_df_2)

column_replacer(gk_df_2)

In [23]:
# You can see here that the df no longer has a multi-level index and the repeated column names have a number attached 
# so that they are not named the same. 
attack_df_2

,Player,#,Nation,Pos,Age,Min,Gls,Ast,PK,PKatt,...,Cmp,Att_1,Cmp%,Prog_2,Carries,Prog_3,Succ,Att_4,game_id,team
0,Alexandre Lacazette,9.0,fr FRA,"FW,LW",26-075,90.0,1,0,0,0,...,19,24,79.2,1,24,2,1,1,1,Arsenal
1,Danny Welbeck,23.0,eng ENG,"AM,RW",26-258,75.0,1,0,0,0,...,16,21,76.2,2,25,6,3,4,1,Arsenal
2,Theo Walcott,14.0,eng ENG,RW,28-148,15.0,0,0,0,0,...,1,2,50.0,0,2,1,0,0,1,Arsenal
3,Mesut Özil,11.0,de GER,AM,28-300,90.0,0,0,0,0,...,69,85,81.2,3,84,15,2,3,1,Arsenal
4,Granit Xhaka,29.0,ch SUI,DM,24-318,90.0,0,2,0,0,...,83,102,81.4,15,89,13,0,0,1,Arsenal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,Alfie Mawson,6.0,eng ENG,CB,23-205,90.0,0,0,0,0,...,34,41,82.9,0,30,1,0,0,5,Swansea City
11,Federico Fernández,33.0,ar ARG,"CB,RB",28-172,90.0,0,0,0,0,...,40,51,78.4,2,44,2,0,0,5,Swansea City
12,Kyle Naughton,26.0,eng ENG,"RB,RM",28-274,90.0,0,0,0,0,...,38,49,77.6,4,27,1,0,0,5,Swansea City
13,Łukasz Fabiański,1.0,pl POL,GK,32-116,90.0,0,0,0,0,...,21,32,65.6,0,20,0,0,0,5,Swansea City


## Rename First Scrape Dataframe
For the first scrape I change the name, this is so that later on, before I concatenate the other leagues to the main DataFrame, I can check for any mistakes in the scrape and makes it easier to concatenate later on. 

In [25]:
att_df = attack_df_2
def_df = defence_df_2
gk_df = gk_df_2
big_meta_df = meta_df

In [ ]:
frames = [gk_df, gk_df_2]
gk_df = pd.concat(frames)

frames = [def_df, defence_df_2]
defence_df = pd.concat(frames)

frames = [att_df, attack_df_2]
att_df = pd.concat(frames)

frames = [big_meta_df, meta_df]
big_meta_df = pd.concat(frames)

In [ ]:
att_df.to_csv('attack.csv')
def_df.to_csv('defence.csv')
gk_df.to_csv('gk.csv')
big_meta_df.to_csv('meta.csv')

### I then repeat these steps for each league that I scrape 
At each stage I save to CSV in order to. 

After Scraping all the leagues and gethering all the data I need you end up with a table like this. 

In [35]:
attack = pd.read_csv('att_show.csv')

attack.drop(columns=['Unnamed: 0', 'Unnamed: 0.1','Unnamed: 0.1.1'], inplace=True)
attack

,Player,#,Nation,Pos,Age,Min,Gls,Ast,PK,PKatt,...,Cmp,Att_1,Cmp_pct,Prog_2,Carries,Prog_3,Succ,Att_4,Game_id,Team
0,Alexandre Lacazette,9.0,fr FRA,"FW,LW",26-075,90.0,1,0,0,0,...,19.0,24.0,79.2,1.0,24.0,2.0,1.0,1.0,1,Arsenal
1,Danny Welbeck,23.0,eng ENG,"AM,RW",26-258,75.0,1,0,0,0,...,16.0,21.0,76.2,2.0,25.0,6.0,3.0,4.0,1,Arsenal
2,Theo Walcott,14.0,eng ENG,RW,28-148,15.0,0,0,0,0,...,1.0,2.0,50.0,0.0,2.0,1.0,0.0,0.0,1,Arsenal
3,Mesut Özil,11.0,de GER,AM,28-300,90.0,0,0,0,0,...,69.0,85.0,81.2,3.0,84.0,15.0,2.0,3.0,1,Arsenal
4,Granit Xhaka,29.0,ch SUI,DM,24-318,90.0,0,2,0,0,...,83.0,102.0,81.4,15.0,89.0,13.0,0.0,0.0,1,Arsenal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194875,Erick Cabaco,3.0,uy URU,CB,25-296,72.0,0,0,0,0,...,9.0,16.0,56.3,2.0,10.0,0.0,0.0,1.0,8220,Getafe
194876,David Timor,24.0,es ESP,CB,31-115,18.0,0,0,0,0,...,8.0,13.0,61.5,2.0,9.0,1.0,0.0,0.0,8220,Getafe
194877,Damián Suárez,22.0,uy URU,RB,32-288,90.0,0,0,0,0,...,25.0,40.0,62.5,5.0,16.0,2.0,0.0,0.0,8220,Getafe
194878,David Soria,13.0,es ESP,GK,27-311,90.0,0,0,0,0,...,22.0,35.0,62.9,0.0,17.0,0.0,0.0,0.0,8220,Getafe
